In [1]:
import os, shutil, glob
import time, requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
os.makedirs(os.path.join("Cellphones Data","Dataset"),exist_ok = True)

In [2]:
def Scrape_Product(URLs, Name):
    driver = webdriver.Edge()

    try:
        URL = URLs
        driver.get(URL)
        
        def close_popups():
            while True:
                try:
                    close_button = WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.modal-close.is-large"))
                    )
                    close_button.click()
                    time.sleep(0.5)
                except Exception:
                    break

            while True:
                try:
                    close_promo_button = WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.cancel-button-top"))
                    )
                    close_promo_button.click()
                    time.sleep(0.5)
                except Exception:
                    break
        
        while True:
            try:
                close_popups()            
                show_more_button = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.button.btn-show-more.button__show-more-product"))
                )
                show_more_button.click()
                time.sleep(2)
            except Exception as e:
                break

        Page = driver.page_source
        Soup = BeautifulSoup(Page, "lxml")
        
        # Tìm tất cả tên và giá sản phẩm
        Product_Names = Soup.find_all("div", class_="product__name")
        Product_Prices = Soup.find_all("p", class_="product__price--show")

        Product_Information = {}

        # Kiểm tra xem số lượng tên sản phẩm và giá có khớp không
        if len(Product_Names) == len(Product_Prices):
            for name, price in zip(Product_Names, Product_Prices):
                product_name = name.get_text(strip=True)
                product_price = price.get_text(strip=True).replace("₫", "").replace(".", "").strip()
                Product_Information[product_name] = product_price
        else:
            print("Số lượng tên sản phẩm và giá sản phẩm không khớp!")

        for product, price in Product_Information.items():
            print(f"Tên sản phẩm: {product} - Giá: {price}")

    finally:
        # Đóng trình duyệt
        driver.quit()
    
    df = pd.DataFrame(list(Product_Information.items()), columns=["Tên sản phẩm", "Giá"])
    df.to_csv(f"{Name}.csv", header=True, index=False)
        


In [3]:
def Scrape_Product_URL():
    URL = "https://cellphones.com.vn/"
    Homepage = requests.get(URL).text
    Soup = BeautifulSoup(Homepage, "lxml")

    Links = Soup.find_all("a", class_="multiple-link")

    Pages = {}
    for link in Links:
        Page_Name = link.get_text(strip=True).replace(",", "")
        Page_URL = link.get("href")
        Pages[Page_Name] = Page_URL

    return Pages

In [4]:
def Scrape_Per_URL():
    Links = Scrape_Product_URL()
    for Num in Links:
        Scrape_Product(Links[Num], Num)
        shutil.move(f"{Num}.csv", os.path.join("Cellphones Data","Dataset"))
Scrape_Per_URL()   

In [7]:
glob.glob(os.path.join("Cellphones Data", "Dataset","*.csv"))
for Files in glob.glob(os.path.join("Cellphones Data", "Dataset","*.csv")):
    File = pd.read_csv(Files)
    Processed_Prices = []
    for price in File['Giá']:
        num = price.replace("đ", "").strip()
        Processed_Prices.append(num)
        
    File['Giá'] = Processed_Prices
    File.to_csv(Files, index=False, encoding='utf-8-sig')